In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomRotation

import pandas as pd

### Tensorflow Dataset 구성 (10점)

- `keras.utils.image_dataset_from_directory` 함수를 활용하여 저장된 이미지 파일로부터 tensorflow Dataset을 생성
- Tensorflow Dataset은 메모리에 상주하는 이미지의 수를 자동으로 조정함으로써 성능 최적화를 이끌어낼 수 있음.

In [ ]:
# 파일 디렉토리 
train_data_path = "./dataset/train"
test_data_path = "./dataset/test"
val_data_path = "./dataset/validation"

image_size = 150
batch_size = 32

# Tensorflow Dataset 생성 
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  train_data_path,
  labels = "inferred", # 레이블을 디렉토리 구조로부터 추론(infer)하도록 지시
  label_mode = "binary",
  color_mode = "rgb",
  batch_size = batch_size,
  image_size = (image_size, image_size),
  class_names = ['cats', 'dogs'],
  shuffle = True,
  seed = 42 
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  val_data_path,
  labels = "inferred",
  label_mode = "binary",
  color_mode = "rgb",
  batch_size = batch_size,
  image_size = (image_size, image_size),
  class_names = ['cats', 'dogs'],
  shuffle = True,
  seed = 42 
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  test_data_path,
  labels = "inferred",
  label_mode = "binary",
  color_mode = "rgb",
  batch_size = batch_size,
  image_size = (image_size, image_size),
  class_names = ['cats', 'dogs'],
  shuffle = False,
)



Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 12461 files belonging to 2 classes.


### 데이터 전처리 (10점)
- 이미지의 size를 (image_size, image_size)로 일괄 조정하였는데, 여기에서 RGB 픽셀 값을 [0,1]값으로 Rescaling

### 데이터 증강 (10점)
- `keras`의 `RandomFlip()`, `RandomRotation()` 레이어를 추가

In [ ]:
model = Sequential()

# 이미지 픽셀 값을 0에서 1사이의 범위로 정규화하고, 모델의 첫 번째 레이어에 입력 이미지의 형태 전달 
model.add(Rescaling(1./255, input_shape = (image_size, image_size, 3)))
# horizontal을 통해 수평 반전 적용
model.add(RandomFlip('horizontal'))
#  [ −0.1×2𝜋, 0.1×2𝜋] 범위 내에서 랜덤하게 회전
model.add(RandomRotation(0.1))

c:\Users\user\anaconda3\envs\kerasenv\lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### 모델 정의 (20점)

A. Convolutional Layers
- CNN 레이어는 총 3개 사용하며, 순서대로 32, 64, 128개의 필터를 사용합니다.
- 모든 CNN 레이어의 필터 크기는 (3,3) 입니다.
- 모든 CNN 레이어의 활성화 함수는 relu 입니다.
- 각 CNN 레이어 뒤에 (2,2) 크기의 MaxPooling2D 레이어를 추가합니다.

B. Fully-connected Layers
- Convolutional Layers의 최종 출력값을 입력받을 수 있도록 Flatten() 레이어를 처음에 추가합니다.
- Dense Layer는 총 2개 사용합니다.
- 은닉층의 차원은 512이며, relu 활성화 함수를 사용합니다.
- 출력층의 차원은 1이며, sigmoid 활성화 함수를 사용합니다.
- 두 은닉층 사이에 Dropout Layer를 추가합니다. (Dropout 확률은 50%)

In [10]:
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

# 최종 출력값을 위해 평탄화
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 150, 150, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    18,940,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,034,177 (72.61 MB)

 Trainable params: 19,034,177 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(
  loss = "binary_crossentropy", 
  optimizer = tf.optimizers.Adam(learning_rate = 1e-4),
  metrics = ["accuracy"])

### 모델 학습 (20점)

- epochs는 30으로 설정합니다.
- 학습 데이터로는 train_dataset을 사용합니다.
- 검증 데이터로는 val_dataset을 사용합니다.
- 미리 정의된 callbacks 배열을 콜백 함수로 등록합니다.

In [14]:
callbacks = [
  tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
]

hist = model.fit(
  train_dataset,
  validation_data = val_dataset,
  epochs = 50,
  callbacks = callbacks,
)

Epoch 1/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 96s 151ms/step - accuracy: 0.6056 - loss: 0.6566 - val_accuracy: 0.7272 - val_loss: 0.5410
Epoch 2/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 95s 153ms/step - accuracy: 0.7340 - loss: 0.5334 - val_accuracy: 0.7546 - val_loss: 0.4973
Epoch 3/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 94s 151ms/step - accuracy: 0.7560 - loss: 0.4926 - val_accuracy: 0.7930 - val_loss: 0.4520
Epoch 4/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 95s 151ms/step - accuracy: 0.7775 - loss: 0.4660 - val_accuracy: 0.8004 - val_loss: 0.4340
Epoch 5/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 94s 151ms/step - accuracy: 0.7904 - loss: 0.4419 - val_accuracy: 0.8222 - val_loss: 0.4065
Epoch 6/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 94s 150ms/step - accuracy: 0.8057 - loss: 0.4243 - val_accuracy: 0.8198 - val_loss: 0.4048
Epoch 7/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 94s 150ms/step - accuracy: 0.8154 - loss: 0.4087 - val_accuracy: 0.8284 - val_loss: 0.3839
Epoch 8/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 95s 152ms/step - accuracy: 0.8215 - loss: 0

### 모델 평가 (30점)

In [15]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"test loss : {test_loss:.3f} | test_acc : {test_acc:.3f}")

390/390 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - accuracy: 0.8700 - loss: 0.2963
test loss : 0.279 | test_acc : 0.881
